![neelwafurat logo](https://www.neelwafurat.com/graphics/common/NWF_logo.png)

# Neelwafurat website books scrapping
**Note: All the links were collected on 11-02-2023**

In this notebook we will scrape all the books data in neelwafurat website to be used later to get books ISBN

In [ ]:
from bs4 import BeautifulSoup
import numpy as np
import threading
import requests
import random
import csv

In [ ]:
USER_AGENTS = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3', 
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36', 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36', 
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:54.0) Gecko/20100101 Firefox/54.0', 
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36', 
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36', 
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36', 
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko', 
    'Mozilla/5.0 (Windows NT 10.0; rv:78.0) Gecko/20100101 Firefox/78.0', 
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36', 
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15', 
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36', 
    'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36 OPR/43.0.2442.991', 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:65.0) Gecko/20100101 Firefox/65.0', 
    'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:52.0) Gecko/20100101 Firefox/52.0', 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36', 
    'Mozilla/5.0 (X11; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0', 
    'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1', 
    'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36', 
    'Mozilla/5.0 (Windows NT 10.0; rv:91.0) Gecko/20100101 Firefox/91.0', 
    'Mozilla/5.0 (Linux; Android 8.0.0; Pixel 2 XL Build/OPD1.170816.004) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Mobile Safari/537.36', 
    'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:54.0) Gecko/20100101 Firefox/54.0', 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36', 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36', 
    'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1 Mobile/15E148 Safari/604.1', 
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/11.1.2 Safari/605.1.15', 
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36', 
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36', 
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36', 
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.106 Safari/537.36', 
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36', 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.109 Safari/537.36', 
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/601.7.7 (KHTML, like Gecko) Version/9.1.2 Safari/601.7.7'
]

In [ ]:
def get_page(url):
    # request to get the company data
    headers = {
        'User-Agent': random.choice(USER_AGENTS),
        "Accept-Encoding": "*",
        "Connection": "keep-alive"
    }
    response = requests.get(url, headers=headers)
    return BeautifulSoup(response.text, 'html.parser') 

In [ ]:
def collect_books_ISBN(pages, thread_number, url):
    number_of_pages = len(pages) - 1
    for index, page in enumerate(pages):
        print(f'Thread #{thread_number}: Start Index {index}/{number_of_pages}')
        url = url + str(page)
        page = get_page(url)
        books = page.find_all('div', {'class': 'col-xs-6 col-s-4 col-lg-3 col-xxl-3 col-3xl-2'})

        for book in books:
            book_info = book.find('div', class_='pro-info')
            link = book_info.find('a').get('href')
            title = book_info.find('a').text

            try:
                price = book_info.find('b', class_='ourprice').text
            except Exception:
                price = None

            my_books.append([title, price, link])


In [ ]:
# all the generas and book lists in the site
lists = {
    '01': {'url':'https://www.neelwafurat.com/browse1.aspx?search=books&ddmSubject=01&orderby=rdate&dir=desc&subcat=01&Page=', 'pages': 17},
    '02': {'url':'https://www.neelwafurat.com/browse1.aspx?search=books&ddmSubject=02&orderby=rdate&dir=desc&subcat=01&Page=', 'pages': 21},
    '03': {'url':'https://www.neelwafurat.com/browse1.aspx?search=books&ddmSubject=03&orderby=rdate&dir=desc&subcat=01&Page=', 'pages': 10},
    '04': {'url':'https://www.neelwafurat.com/browse1.aspx?search=books&ddmSubject=04&orderby=rdate&dir=desc&subcat=01&Page=', 'pages': 71},
    '05': {'url':'https://www.neelwafurat.com/browse1.aspx?search=books&ddmSubject=05&orderby=rdate&dir=desc&subcat=01&Page=', 'pages': 371},
    '06': {'url':'https://www.neelwafurat.com/browse1.aspx?search=books&ddmSubject=06&orderby=rdate&dir=desc&subcat=01&Page=', 'pages': 5},
    '07': {'url':'https://www.neelwafurat.com/browse1.aspx?search=books&ddmSubject=07&orderby=rdate&dir=desc&subcat=01&Page=', 'pages': 2},
    '08': {'url':'https://www.neelwafurat.com/browse1.aspx?search=books&ddmSubject=08&orderby=rdate&dir=desc&subcat=01&Page=', 'pages': 240},
    '09': {'url':'https://www.neelwafurat.com/browse1.aspx?search=books&ddmSubject=09&orderby=rdate&dir=desc&subcat=01&Page=', 'pages': 1},
    '10': {'url':'https://www.neelwafurat.com/browse1.aspx?search=books&ddmSubject=10&orderby=rdate&dir=desc&subcat=01&Page=', 'pages': 588},
    '11': {'url':'https://www.neelwafurat.com/browse1.aspx?search=books&ddmSubject=11&orderby=rdate&dir=desc&subcat=01&Page=', 'pages': 9},
    '12': {'url':'https://www.neelwafurat.com/browse1.aspx?search=books&ddmSubject=12&orderby=rdate&dir=desc&subcat=01&Page=', 'pages': 17},
    '13': {'url':'https://www.neelwafurat.com/browse1.aspx?search=books&ddmSubject=13&orderby=rdate&dir=desc&subcat=01&Page=', 'pages': 12},
    '14': {'url':'https://www.neelwafurat.com/browse1.aspx?search=books&ddmSubject=14&orderby=rdate&dir=desc&subcat=01&Page=', 'pages': 4},
    '15': {'url':'https://www.neelwafurat.com/browse1.aspx?search=books&ddmSubject=15&orderby=rdate&dir=desc&subcat=01&Page=', 'pages': 1},
    '16': {'url':'https://www.neelwafurat.com/browse1.aspx?search=books&ddmSubject=16&orderby=rdate&dir=desc&subcat=01&Page=', 'pages': 1125},
    '17': {'url':'https://www.neelwafurat.com/browse1.aspx?search=books&ddmSubject=17&orderby=rdate&dir=desc&subcat=01&Page=', 'pages': 11},
    '18': {'url':'https://www.neelwafurat.com/browse1.aspx?search=books&ddmSubject=18&orderby=rdate&dir=desc&subcat=01&Page=', 'pages': 5},
    '19': {'url':'https://www.neelwafurat.com/browse1.aspx?search=books&ddmSubject=19&orderby=rdate&dir=desc&subcat=01&Page=', 'pages': 42},
    '20': {'url':'https://www.neelwafurat.com/browse1.aspx?search=books&ddmSubject=20&orderby=rdate&dir=desc&subcat=01&Page=', 'pages': 4},
    '21': {'url':'https://www.neelwafurat.com/browse1.aspx?search=books&ddmSubject=21&orderby=rdate&dir=desc&subcat=01&Page=', 'pages': 144},
    '22': {'url':'https://www.neelwafurat.com/browse1.aspx?search=books&ddmSubject=22&orderby=rdate&dir=desc&subcat=01&Page=', 'pages': 8},
    '23': {'url':'https://www.neelwafurat.com/browse1.aspx?search=books&ddmSubject=23&orderby=rdate&dir=desc&subcat=01&Page=', 'pages': 1},
    '24': {'url':'https://www.neelwafurat.com/browse1.aspx?search=books&ddmSubject=24&orderby=rdate&dir=desc&subcat=01&Page=', 'pages': 2},
    '25': {'url':'https://www.neelwafurat.com/browse1.aspx?search=books&ddmSubject=25&orderby=rdate&dir=desc&subcat=01&Page=', 'pages': 55},
    '26': {'url':'https://www.neelwafurat.com/browse1.aspx?search=books&ddmSubject=26&orderby=rdate&dir=desc&subcat=01&Page=', 'pages': 48},
}

In [ ]:
for index, item in lists.items():
    my_books = []
    print('-' * 100)
    print(f'Start INDEX # {index}')
    print('-' * 100)
    pages = [i for i in range(1, item.get('pages') + 1)]
    splited_pages = np.array_split(pages, 8)
    
    # start runing the threads and gave each one the df that it will scrape
    threads = []
    for i, splited_page in enumerate(splited_pages):
        thread = threading.Thread(target=collect_books_ISBN, kwargs={'pages': splited_page, 'thread_number': str(i + 1), 'url': item.get('url')})
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()
        
    with open(f'my_books_lib_neelwafurat_arabic_{index}.csv', 'w', encoding='utf-8-sig', newline="") as file:
        writer = csv.writer(file)
        writer.writerow(['Name', 'Price', 'Link'])
        for book in my_books:
            writer.writerow(book)